In [1]:
import requests, os
from bs4 import BeautifulSoup
from src.downloader import download

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text
    

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image


def check_word(word):
    def check(url):
        return (word in url)
    return check


In [19]:
keyword = "home speaker"
page = "1"
url = "https://www.amazon.fr/s?k={}&page={}&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=sr_pg_{}".format(keyword, page, page)

soup = BeautifulSoup(get_html(url))
# print(soup.prettify())
for image_url in list(filter(check_word("_AC_UL320_"), get_image(soup))):
    download(image_url, "Products")